In [ ]:
DESCRIPTION = """Convert an atomate2 TaskDocument of VASP data to an interpretable set of features"""

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

doc:
    type: Dict
    description: Atomate2 TaskDocument to be converted to a set of interpretable features
    value: {}
        
author:
    type: Tag
    description: Author name for this vasp input
    value: ""
        
dataset:
    type: Tag
    description: Dataset associated with this vasp input
    value: ""

In [ ]:
EXTRA_FILES = []

In [ ]:
from simtool import getValidatedInputs

defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

In [ ]:
from simtool import getParamsFromDictionary

inputParameters = getParamsFromDictionary(INPUTS,globals())

In [ ]:
%%yaml OUTPUTS

structure:
    type: Dict
    description: pymatgen Structure object, containing lattice vectors and atomic positions

composition: # TODO: ask saswat about ways to make composition easily queryable
    type: Dict
    description: chemical composition of the compound 

lattice_parameters:
    type: Array
    description: a, b, c lattice parameters of unit cell (Å)

lattice_angles:
    type: Array
    description: α, β, γ angles of the unit cell (degrees)
        
energy:
    type: Number
    description: total energy of the system (eV)

stress:
    type: Array
    description: stress tensor of the system (kbar) # TODO: verify units from atomate

forces:
    type: Array
    description: forces on each atom (eV/Å) # TODO: verify units from atomate
        
max_force:
    type: Number
    description: maximum force reported during the simulation (eV/Å)
        
rms_force:
    type: Number
    description: RMS force reported during the simulation (eV/Å)

KPOINTS:
    type: Array
    description: number of k-points in the x, y, and z directions

ENCUT:
    type: Number
    description: kinetic energy cutoff for the plane wave basis set (eV)
        
XC_functional:
    type: Tag
    description: choice of exchange-correlation functional used, read from VASP's GGA tag
        
pseudopotential:
    type: Tag
    description: choice of pseudopotential used


In [ ]:
# Import required libraries
import numpy as np
from pymatgen.core import Structure


In [ ]:
# Validate inputs

# check if keys in input_dict are the same keys as in an atomate2 task document (atomate2 version 0.0.7)
doc_keys = ['composition', 'input', 'output', 'structure', 'calcs_reversed']


if any(key not in doc.keys() for key in doc_keys):
    raise Exception("This doesn't appear to be an atomate2 TaskDocument dict. \nSee the vaspingestor Sim2L documentation for details")

In [ ]:
# structure information
structure = doc['output']['structure']
composition = doc['composition']
struct = Structure.from_dict(structure)
lattice_parameters = np.array(struct.lattice.abc)
lattice_angles = np.array(struct.lattice.angles)

# simulation outputs
energy = doc['output']['energy']
stress = doc['output']['stress']
forces = doc['output']['forces']

max_force = np.max(forces)
rms_force = np.sqrt(np.mean([f**2 for f in np.array(forces).flatten()]))

# simulation parameters
KPOINTS = doc['calcs_reversed'][-1]['input']['kpoints']['kpoints'][0]
ENCUT = doc['calcs_reversed'][-1]['input']['incar']['ENCUT']

XC_functional = doc['input']['parameters']['GGA'] 
pseudopotential = doc['input']['pseudo_potentials']['pot_type']


In [ ]:
from simtool import DB
db = DB(OUTPUTS)

In [ ]:
db.save('structure',structure)
db.save('composition',composition)
db.save('lattice_parameters',lattice_parameters)
db.save('lattice_angles',lattice_angles)
db.save('energy',energy)
db.save('stress',stress)
db.save('forces',forces)
db.save('max_force',max_force)
db.save('rms_force',rms_force)
db.save('KPOINTS',KPOINTS)
db.save('ENCUT',ENCUT)
db.save('XC_functional',XC_functional)
db.save('pseudopotential',pseudopotential)